# Problem Set 5
Designed by Ben Usman, Sarah Adel Bargal, and Brian Kulis, with help from Kun He and Kate Saenko.

This assignment will introduce you to:

1. implementation of RNN and LSTM
2. training your implemented LSTM
3. deriving multi-modal Restricted Boltzmann Machines

## Part 1: RNN Example

(10 points)

In this example we train an RNN to infer the parameters of a simple dynamical system.
First, we simulate a dynamical system with known parameters (random numbers), and use it to generate outputs. Then, we train an RNN on the generated datapoints, attempting to infer the original parameters.
You are expected to run and study the provided code, which will be helpful for the second part (implementing LSTM).

1. We define a discretized [dynamical system](https://en.wikipedia.org/wiki/Dynamical_system).
At each discrete time $t$, the system observes input $x_t$. The system maintains some "state" $h_t$, which will be updated over time.
Specifically, the states are updated by the following rule: $h_t = \max(0, 1-(Wx_t+h_{t-1}))$, where $W$ is a parameter matrix.
2. In this example, the input data $\{x_t\}$ is randomly generated, and the weight matrix $W$ is also drawn randomly. The system starts from an initial hidden state $h_0$, and runs for $t=1,\ldots,T$.
3. Given the sequence of states $\{h_1,\ldots,h_T\}$, we would like to infer the weights $W$.

To start with, the following code segment generates and displays the data.
Refer to `data_generator.py` for details of data generation.

In [ ]:
# Here we present sample data generated 
import numpy as np
import data_generator
import matplotlib.pyplot as plt

(h_0, w), x, h = data_generator._build_rnn_testdata_matrix()

print('variable name, shape, min, max: ')
for v, name in zip([h_0, w, x], ['h0', 'w', 'x']):
    print(name, v.shape, np.min(v), np.max(v))
norm_x_t = np.sum(x*2, axis=1)
plt.title('||x(t)||')
plt.plot(np.arange(norm_x_t.shape[0]), norm_x_t)
plt.show()

We model the dynamical system with an RNN. We would like the state generated by the RNN to match the actual observed, and we use L2 loss for this purpose.
This RNN is a *regression* model since it outputs real values.

Below, `build_rnn_regression_model()` gives the model definition, and `train_rnn_with_noise()` generates a batch of data and then runs training on it. 
We corrupt the data generation process with noise, and let the dimensionality of the state $h$ be a free parameter. Therefore, the training function takes two input arguments: `noise_level` and `n_hidden_dim`.   You will later see how varying them affects reconstruction quality.

In [ ]:
import datetime
import os
import numpy as np
import tensorflow as tf
from __future__ import print_function

#############################################################################
# RNN model graph
def build_rnn_regression_model(shape):
    # shape is dict with keys:
    # n_steps_per_batch, n_hidden_dim, n_input_dim
    with tf.Graph().as_default() as g:
        # inputs to the dynamical system
        X = tf.placeholder(tf.float32,
                           [None, shape['n_steps_per_batch'], shape['n_input_dim']])
        # observed state from the dynamical system
        y = tf.placeholder(tf.float32, [None, shape['n_hidden_dim']])
        
        with tf.variable_scope('weights'):
            # weight matrix
            w = tf.get_variable('w', [shape['n_input_dim'], shape['n_hidden_dim']])
            # initial state
            h_0 = tf.get_variable('h_0', [shape['n_hidden_dim']])
            
        # for t = 1 to T, update state 
        h_t = h_0
        for t in range(shape['n_steps_per_batch']):
            x_t = X[:, t, :]
            h_t = tf.maximum(0.0, 1 - (tf.matmul(x_t, w) + h_t))
        
        # loss: L2
        loss = tf.nn.l2_loss(h_t - y, name='loss')
        train_op = tf.train.AdamOptimizer(0.1).minimize(loss)
        summ = tf.summary.scalar('loss_sum_%dd' % shape['n_hidden_dim'], loss)
        
    return {'inputs': [X, y], 'loss': loss, 'train_op': train_op, 'summ': summ,
            'weights': {'w': w, 'h_0': h_0}, 'graph': g}

#############################################################################
# Main train loop for an RNN regression model
# 
# This takes synthetic data generated by data_generator.build_dataset()
# the weight matrix W is then inferred with back-prop 
def train_rnn_with_noise(noise_level, n_hidden_dim):
    # generate data
    shapes = dict(n_hidden_dim=n_hidden_dim, n_input_dim=15, n_steps_per_batch=100)
    rnn_dataset = data_generator.build_dataset('rnn', noise=noise_level, **shapes)
    (h0_true, w_true), batched_data = rnn_dataset  # "true" weights
    # build RNN model
    model = build_rnn_regression_model(shapes)
    
    #logdir = './tensorboard/rnn_demo'  # if on Windows
    logdir = '/tmp/tensorboard/rnn_demo'  # if on Unix
    try:
        os.makedirs(logdir)
    except os.error:
        pass
    # If you want to see the plots, run tensorboard:
    # $ tensorboard --logdir=[your_logdir]
    #
    # If you use SCC, you can forward the 6006 port from the cluster 
    # to your local machine via:
    # $ ssh [SCC_cluster_name] -L 6006:localhost:6006
    time_now = datetime.datetime.now().strftime("%d-%b-%H-%M-%S")
    run_name = 'hidden=%d_noise=%.2f' % (n_hidden_dim, noise_level)
    sum_path = os.path.join(logdir, run_name + '_' + time_now)
    print(sum_path)
    max_iter_i = 0
    with model['graph'].as_default() as g, tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sum_writer = tf.summary.FileWriter(sum_path, g)
        for epoch_i in range(50):
            loss_val, w_dist, h0_dist, iter_i = None, None, None, None
            for iter_i, data_batch in enumerate(batched_data):
                max_iter_i = max(iter_i, max_iter_i)
                global_step = epoch_i*max_iter_i+iter_i
                
                # run training step
                train_feed_dict = dict(zip(model['inputs'], data_batch))
                to_compute = [model['train_op'], model['summ'], model['loss'],
                              model['weights']['w'], model['weights']['h_0']]
                _, summ, loss_val, w_val, h0_val = sess.run(to_compute, train_feed_dict)
                
                # compute reconstruction error
                w_err = np.linalg.norm(w_true-w_val)
                h0_err = np.linalg.norm(h0_true-h0_val)
                
                # for tensorboard
                sum_writer.add_summary(summ, global_step)
                sum_writer.add_summary(tf.Summary(value=[
                    tf.Summary.Value(tag="w_true_dist_%dd" % n_hidden_dim,
                                     simple_value=w_dist),
                ]), global_step)
                sum_writer.add_summary(tf.Summary(value=[
                    tf.Summary.Value(tag="h_true_dist_%dd" % n_hidden_dim,
                                     simple_value=h0_dist),
                ]), global_step)
                sum_writer.flush()
                
            print('epoch %d, loss=%g, w_err=%g'%(epoch_i, loss_val, w_err))
            if global_step > 200: 
                break  # just train for 200 iterations

### Q1.
Now test the RNN wth varying noise levels and hidden dimensionalities.
- For each combination of `n_hidden_dim` and `noise_level`, report the reconstruction error (`w_err`).
- Describe how the hidden dimentionality and the noise level influence reconstruction quality. And briefly explain why.

In [ ]:
# Experimenting with different data noise levels and hidden dimentionalities
# We are lucky to know the true hidden dimentionality in our simultaion
for n_hidden_dim in [10, 100, 1000]:
    for noise_level in [0, 0.1, 0.5]:
        print(n_hidden_dim, noise_level)
        train_rnn_with_noise(noise_level, n_hidden_dim)

## Part 2: LSTM implementation

(40 points)

Now let's attempt to recover the weights in dynamical system simulated with an LSTM.  Although LSTMs are already implemented in TensorFlow ([tutorial here](https://www.tensorflow.org/tutorials/recurrent)) ([source here](https://github.com/tensorflow/tensorflow/blob/efe5376f3dec8fcc2bf3299a4ff4df6ad3591c88/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py#L264)), you will be implementing a simple LSTM from scratch using Tensorflow in this part. 


### Q2. Implement an LSTM 
Implement an LSTM model in an analogous way,  in functions `build_lstm_regression_model()` and `train_lstm_with_noise()` below.
The RNN regression implementation above should give you some ideas.
We already provided an LSTM version of the dynamical system generator in the dataset generator code.

- Specifically, you can follow and implement Eq. 1-6 from [this link](http://deeplearning.net/tutorial/lstm.html) in `build_lstm_regression_model()`. 
You may simplify your code by concatenating $x$ and $h$.
- Afterwards, implement `train_lstm_with_noise()` to train the LSTM and recover the parameters. Compute the reconstruction errors for $W_c$ and $U_c$, which are the parameters used in Eq. 2 in the link.
- For each combination of hidden dimension and noise level, report the reconstruction error (`w_err`, `u_err`) you get from the LSTM.

Note:
- The weights might not get reconstructed correctly in the LSTM case even without noise. (Why?)
Nevertheless, the loss should decrease.

In [ ]:
def build_lstm_regression_model(shape):
    # shape is dict with keys:
    # n_steps_per_batch, n_hidden_dim, n_input_dim
    with tf.Graph().as_default() as g:
        # inputs
        X = tf.placeholder(tf.float32,
                           [None, shape['n_steps_per_batch'], shape['n_input_dim']])
        # observed outputs
        y = tf.placeholder(tf.float32, [None, shape['n_hidden_dim']])
        
        #################################################################
        ####################  PUT YOUR CODE HERE   ######################
        # define the parameters in the LSTM (scope: weights)
        # and carry out the computation according to Eq. 1-6 in the link
        with tf.variable_scope('weights'):
            pass
        output = None  # put your result in variable 'output'
        #################################################################
        
        # loss and train_op
        loss = tf.nn.l2_loss(output - y, name='loss')
        train_op = tf.train.AdamOptimizer(0.1).minimize(loss)
        summ = tf.summary.scalar('loss_sum_%dd' % shape['n_hidden_dim'], loss)

    return {'inputs': [X, y], 'loss': loss, 'train_op': train_op, 'summ': summ,
            'weights': {'w_c': w_c, 'u_c': u_c}, 
            'graph': g}
    
    
def train_lstm_with_noise(noise_level, n_hidden_dim):
    # generate data and random weights
    shapes = dict(n_hidden_dim=n_hidden_dim, n_input_dim=15, n_steps_per_batch=100)
    weights, batched_data = data_generator.build_dataset('lstm', noise=noise_level, **shapes)
    w_c, u_c = weights[3], weights[7]  # the "true" weights to recover: Wc & Uc (in Eq.2)
    
    # this is the function you implemented
    model = build_lstm_regression_model(shapes)
    
    #logdir = './tensorboard/lstm_demo'  # if on Windows
    logdir = '/tmp/tensorboard/lstm_demo'  # if on Unix
    try:
        os.makedirs(logdir)
    except os.error:
        pass
    time_now = datetime.datetime.now().strftime("%d-%b-%H-%M-%S")
    run_name = 'hidden=%d_noise=%.2f' % (n_hidden_dim, noise_level)
    sum_path = os.path.join(logdir, run_name + '_' + time_now)
    print(sum_path)
    
    max_iter_i = 0
    with model['graph'].as_default() as g, tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sum_writer = tf.summary.FileWriter(sum_path, g)
        for epoch_i in range(10):  # 10 epochs by default, feel free to change
            loss_val, w_err, u_err, iter_i = None, None, None, None
            for iter_i, data_batch in enumerate(batched_data):
                max_iter_i = max(iter_i, max_iter_i)
                global_step = epoch_i*max_iter_i+iter_i
                
                ###############################################################
                ###################   PUT YOUR CODE HERE   ####################
                train_feed_dict = None  # define your train_feed_dict
                to_compute = []  # the things to compute, including at least:
                                 # loss_val, w_err, u_err
                outputs = sess.run(to_compute, train_feed_dict)
                w_err = None  # compute them and report
                u_err = None
                # [optional] you can also do early stopping, e.g. 300 iterations
                # if global_step > 300:
                #    break
                # use sum_writer for tensorboard, see train_rnn_with_noise()
                sum_writer.add_summary(...)
                sum_writer.flush()
                ###############################################################

In [ ]:
for n_hidden_dim in [10, 100, 1000]:
    for noise_level in [0, 0.1, 0.5]:
        print(n_hidden_dim, noise_level)
        train_lstm_with_noise(noise_level, n_hidden_dim)

## Part 3: Neural Tolstoy Model (a character-prediction LSTM)

(20 points)

In this part you will train a character prediction LSTM that predicts the next character given previous characters. 
The training data is from the book "War and Peace" by Leo Tolstoy. 

### Q3.
Compared to the LSTM you implemented in the previous part, the main difference in the character prediction LSTM is that it predicts *discrete* outputs (characters, or their indices in the vocabulary), therefore it is a classification model. As you may recall, the usual choice of loss function for classification is softmax + cross entropy.

We have already provided functions for loading the training data. Please define your discrete LSTM in `build_lstm_discrete_prediction_model()`.

Hints: 
- Your LSTM should predict a single character at a time. (That's why it's called a character prediction LSTM.) We are not considering  many-to-many LSTMs here.
- Feeding the previous input into the LSTM should help.

In [ ]:
import tolstoy_reader

def get_default_gpu_session(fraction=0.333):
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True
    config.gpu_options.per_process_gpu_memory_fraction = fraction
    return tf.Session(config=config)

def run_tolstoy_train(n_hid):
    # generate data
    btg, map_dict, backmap_dict = \
        tolstoy_reader.batch_tolstoy_generator(batch_size=200, seq_size=100)
    shape = dict(n_steps_per_batch=100, n_unique_ids=len(map_dict), n_hidden_dim=n_hid)
    # define LSTM
    model = build_lstm_discrete_prediction_model(shape)
    
    #logdir = './tensorboard/tolstoy'  # if on Windows
    logdir = '/tmp/tensorboard/tolstoy'  # if on Unix
    try:
        os.makedirs(logdir)
    except os.error:
        pass
    time_now = datetime.datetime.now().strftime("%d-%b-%H-%M-%S")
    run_name = 'hidden=%d' % n_hid
    sum_path = os.path.join(logdir, run_name + '_' + time_now)
    print(sum_path)
    max_iter_i = 0
    with model['graph'].as_default() as g, get_default_gpu_session(0.9) as sess:
        sess.run(tf.global_variables_initializer())
        sum_writer = tf.summary.FileWriter(sum_path, g)
        for epoch_i in range(10):
            for iter_i, data_batch in enumerate(btg):
                max_iter_i = max(iter_i, max_iter_i)
                global_step = epoch_i*max_iter_i+iter_i
                
                # run training step
                train_feed_dict = dict(zip(model['inputs'], data_batch))
                to_compute = [model['train_op'], model['summ'], model['loss']]
                _, summ, loss_val = sess.run(to_compute, train_feed_dict)
                
                # for tensorboard
                sum_writer.add_summary(summ, global_step)
                sum_writer.flush()
                
                # display loss
                if iter_i % 100 == 0:
                    print(loss_val, end=', ')
                if iter_i % 1000:
                    continue
                # test generation
                pred_length = 50
                data_input = next(iter(btg))[0][[0]]
                original_sample = data_input.copy()
                pred_seq = []
                for _ in range(pred_length):
                    pred = sess.run(model['pred'], {model['inputs'][0]: data_input})
                    pred_seq.append(pred[0])
                    data_input = np.roll(data_input, -1, axis=1)
                    data_input[0, -1] = pred[0]
                print('[%d] Input text:' % (iter_i))
                print(''.join([backmap_dict[x] for x in original_sample[0]]))
                print('[%d] Generated continuation:' % (iter_i))
                print(''.join([backmap_dict[x] for x in pred_seq]))
                print(pred_seq)
                print()
                
def build_lstm_discrete_prediction_model(shape):
    # shape is dict with keys:
    # n_steps_per_batch, n_unique_ids, n_hidden_dim
    with tf.Graph().as_default() as g:
        X = tf.placeholder(tf.int64, [None, shape['n_steps_per_batch']])
        y = tf.placeholder(tf.int64, [None])
        
        ################################################################
        ####################   PUT YOUR CODE HERE   ####################
        # define LSTM parameters (scope: weights)
        with tf.variable_scope('weights'):
            pass
        logits = None  # compute logits for each discrete output
        loss = None  # compute loss with respect to (logits, y)
        ################################################################

        # pred, train_op
        pred = tf.argmax(logits, axis=1)
        train_op = tf.train.AdamOptimizer().minimize(loss)
        summ = tf.summary.scalar('loss_summ', loss)

    return {'inputs': [X, y], 'loss': loss, 'train_op': train_op, 'summ': summ,
            'graph': g, 'pred': pred}

Run the LSTM and see what it says! 

In [ ]:
hidden = 200
run_tolstoy_train(hidden)

## Part 4: Multi-modal Restricted Boltzmann Machines

(30 points)

[Relevant reading: Goodfellow chapter 20 through 20.8, in particular 20.2 and 20.3]

For this question, we will consider designing an unsupervised probability distribution  over two modalities.  Suppose we have data consisting of images and their associated captions, so that each input data point may be thought of as the pair $(t,v)$, where $t = (t_1, ..., t_n)$ is the text data and $v = (v_1, ..., v_m)$ is the visual data.  For simplicity, we will consider both as vector inputs (so, for instance, we will not be considering any two-dimensional convolutions for the visual data); the main difference between the two inputs is that the text input is discrete-valued, where we assume that each $t_i$ can take on $k$ different values, and each visual input is real-valued.

Following our discussion of Restricted Boltzmann Machines (RBMs), we will further consider a hidden layer with **binary** hidden units $(h_1, ..., h_d)$, and define a joint probability distribution over an input data point and a hidden layer.  In particular, let us define two matrices $W_t\in\mathbb{R}^{n\times d}$ and $W_v\in\mathbb{R}^{m\times d}$; these matrices correspond to weights between text input and hidden units, and weights between visual input and hidden units, respectively.  In addition, we have the bias vectors $a$, $b$, and $c$, associated with the text, visual, and hidden units, respectively.
The multi-modal RBM is illustrated below.

<img src="MultimodalRBM.png" width="550">

Next we define the joint probability distribution $p(t,v,h)$ over text inputs, visual inputs, and hidden states.  First we let the energy function be: 

$E(v,t,h) = -t^T W_t h - v^T W_v h - a^T t + (0.5 b^Tb - b^T v + 0.5 v^T v) - c^T h$.

Then let the joint probability be defined as:

$p(v,t,h) = \frac{1}{Z}exp(-E(v,t,h)),$

where $Z$ is the normalizing constant that ensures that the probability distribution is properly normalized to sum/integrate to one.

Our goal is to perform maximum likelihood estimation for the parameters $W_t$, $W_v$, and $c$ (we will assume $a$ and $b$ are fixed and known), given a set of $N$ data points $D = \{(t^{(1)},v^{(1)}), ..., (t^{(N)},v^{(N)})\}$.  As is typical in such settings, we assume that the data in $D$ are i.i.d. samples.

Please write your solutions in the cells below, or hand in a hard copy.
For each question you must show your work to receive full credit.

### Q4.1 

Write down the marginal probability for a single data point $(t^{(1)}, v^{(1)})$, namely $p(t^{(1)}, v^{(1)})$. Treating this probability as the likelihood for a single point, write down the log-likelihood for the entire data set $D$ as a function of the parameters $(W_t, W_v,  c)$.

**[Put your answer here]**

### Q4.2 

Write down the partial derivative of $E(t,v,h)$ with respect to each of the parameters, i.e., each entry of $W_t$, each entry of $W_v$, and each entry of $c$. You can also directly write in matrix/vector forms.


**[Put your answer here]**

### Q4.3 

Derive the conditional distributions $p(h | v, t)$, $p(v | h)$ and $p(t | h)$, either in elementwise form or in vector form.
Also establish that $v$ and $t$ are conditionally independent given $h$.

**[Put your answer here]**

### Q4.4

Write down the partial derivative of the log-likelihood for $D$ with respect to each of the parameters.

**[Put your answer here]**

### Q4.5 

Extend the contrastive divergence approach for standard RBMs, as discussed in class, to suggest a gradient ascent procedure for learning the parameters of our multi-modal RBM.

**[Put your answer here]**